In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

from hex import HexGame
from neuralnet import createModel, loadModel
from player import NeuralNetPlayer, RandomPlayer, MCTSPlayer, NeuralMCTSPlayer
from tournament import Tournament
import pickle
from mcts import Mcts
from keras.callbacks import EarlyStopping, ModelCheckpoint
import time
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.layers import Dropout, BatchNormalization
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import RMSprop

tf.config.run_functions_eagerly(True)
%matplotlib qt

In [145]:
boardSize = 7
dataName = f'replayBuffer{boardSize}.pickle'
with open(dataName, "rb") as f:
    replay = pickle.load(f)
print("Loaded", len(replay), "games from replay buffer")

from sklearn.model_selection import train_test_split
X = np.array([x[0] for x in replay]).reshape(len(replay), -1)
y = np.array([x[1] for x in replay]).reshape(len(replay), -1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# calculate baseline accuracy by predicting the most common move on validation set
from collections import Counter
mostCommonMove = Counter([np.argmax(x[1]) for x in replay]).most_common(1)[0][0]
baselineAccuracy = sum([np.argmax(x[1]) == mostCommonMove for x in replay])/len(replay)
print("Baseline accuracy:", baselineAccuracy, "with most common move", mostCommonMove)

Loaded 2450 games from replay buffer
Baseline accuracy: 0.04122448979591837 with most common move 16


In [146]:
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from keras.layers import LeakyReLU

from keras.layers import ELU

from keras.regularizers import l2

from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
import keras.backend as K

from keras.layers import ELU

from keras.optimizers import RMSprop

from keras.optimizers import Nadam

from keras.optimizers import SGD
from keras.layers import Conv2D, Flatten

from keras.layers import Conv2D, Flatten, Reshape

from keras.layers import Conv2D, Flatten, Reshape, MaxPooling2D

def createModel(size):
    input_shape = (size, size, 4)  # Assumes square images with 4 channels
    model = Sequential()
    model.add(Reshape(input_shape, input_shape=(size*size*4,)))  # Reshape the flattened input data
    model.add(Conv2D(size, kernel_size=(3, 3), activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(size*size, activation='softmax', kernel_initializer='he_uniform'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])
    return model

model = createModel(boardSize)
history = model.fit(X_train, y_train, epochs=200, batch_size=16, validation_data=(X_test, y_test), verbose=0)
# find point of best validation accuracy
bestValAcc = max(history.history['val_accuracy'])
print("Best validation accuracy:", round(bestValAcc,4))
# best loss
bestValLoss = min(history.history['val_loss'])
print("Best validation loss:", round(bestValLoss,4))

c:\Users\Jacob\Anaconda3\envs\pytorch\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
c:\Users\Jacob\Anaconda3\envs\pytorch\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Best validation accuracy: 0.1755
Best validation loss: 3.3929


In [144]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [143]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# predict 100 times on state and check time
start = time.time()
for i in range(100):
    model.predict(np.array([state]).reshape(1, -1), verbose=0)
end = time.time()
print("Time per prediction:", (end - start) / 100)

In [ ]:
# predict 100 times on state with call and check time
start = time.time()
for i in range(100):
    # first make state into a tensor
    stateTensor = tf.convert_to_tensor(np.array([state]).reshape(1, -1), dtype=tf.float32)
    model(stateTensor)
end = time.time()
print("Time per prediction with call:", (end - start) / 100)

In [ ]:
# check if both methods give same result
stateTensor = tf.convert_to_tensor(np.array([state]).reshape(1, -1))
print("Predictions equal:", np.array_equal(model.predict(np.array([state]).reshape(1, -1), verbose=0), model(stateTensor)))
# print prediction
print("Prediction:", model(stateTensor))

### Test if NN mcts is better than normal mcts

In [142]:
boardSize = 4
bestModel = loadModel(f'model.{boardSize}')

mctsPlayer = MCTSPlayer(maxIters=100, maxTime=999999999, argmax=True)
nnMctsPlayer = NeuralMCTSPlayer(model=bestModel, maxIters=100, maxTime=999999999, argmax=True)
tournament = Tournament(HexGame, [nnMctsPlayer, mctsPlayer], boardSize=boardSize, plot=True)
tournament.run(10)
tournament.printResults()

iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iterations: 100
iteratio

In [63]:
# test if neuralnet alone is better than mcts
mctsPlayer = MCTSPlayer(maxIters=10000000, maxTime=1, argmax=True)
nnPlayer = NeuralNetPlayer(model=bestModel, argmax=True)
tournament = Tournament(HexGame, [nnPlayer, mctsPlayer], boardSize=boardSize, plot=True)
tournament.run(10)
tournament.printResults()

iterations: 414
iterations: 465
iterations: 618
iterations: 1195
iterations: 1525
iterations: 368
iterations: 399
iterations: 507
iterations: 870
iterations: 1472
iterations: 406
iterations: 523
iterations: 550
iterations: 901
iterations: 1243
iterations: 364
iterations: 390
iterations: 579
iterations: 1423
iterations: 417
iterations: 420
iterations: 781
iterations: 1345
iterations: 366
iterations: 408
iterations: 574
iterations: 1451
iterations: 416
iterations: 531
iterations: 759
iterations: 1235
iterations: 370
iterations: 472
iterations: 718
iterations: 1591
iterations: 397
iterations: 528
iterations: 929
iterations: 1022
iterations: 1345
iterations: 360
iterations: 397
iterations: 551
iterations: 1294
iterations: 399
iterations: 513
iterations: 949
iterations: 1019
iterations: 1359
iterations: 361
iterations: 392
iterations: 482
iterations: 972
iterations: 1538
iterations: 386
iterations: 531
iterations: 796
iterations: 1383
iterations: 365
iterations: 393
iterations: 734
iteratio

### Test critic

In [ ]:
# create data for critic
boardSize = 4
# Let two mcts players play against each other
mctsPlayer1 = MCTSPlayer(maxIters=100, maxTime=1000, argmax=True)
mctsPlayer2 = MCTSPlayer(maxIters=100, maxTime=1000, argmax=True)
tournament = Tournament(HexGame, [mctsPlayer1, mctsPlayer2], boardSize=boardSize, plot=False)
tournament.run(200)
tournament.printResults()

In [ ]:
# get replay buffer from each players
replayBuffer = mctsPlayer1.mcts.replayBuffer + mctsPlayer2.mcts.replayBuffer
print(f'Length of replay buffer: {len(replayBuffer)}')

In [ ]:
X = [r[0] for r in replayBuffer]
y = [r[-1] for r in replayBuffer]
X = np.array(X).reshape(len(X), -1)
y = np.array(y).reshape(-1, 1)
# for all -1's in y set it to 0
y[y == -1] = 0

In [ ]:
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import RMSprop

# CRITIC
def createCriticModel(size):
    model = Sequential()
    model.add(Dense(size*size, input_dim=size*size+1, activation='relu'))
    model.add(BatchNormalization())
    # dropout layer
    model.add(Dropout(0.3))
    model.add(Dense(size, activation='relu'))
    model.add(BatchNormalization())
    # Add another Dense layer
    model.add(Dense(size, activation='relu'))
    model.add(BatchNormalization())
    # Add another dropout layer
    model.add(Dropout(0.3))
    # final layer is a singular value from 0 to 1
    model.add(Dense(1, activation='sigmoid', kernel_initializer='he_uniform'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Early stopping callback to stop training when validation accuracy has stopped improving
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, mode='max', restore_best_weights=True)
# Model checkpoint callback to save the best model weights during training
model_checkpoint = ModelCheckpoint('best_model_weights.h5', monitor='val_accuracy', save_best_only=True, verbose=0, mode='max')

critic = createCriticModel(boardSize)
# split into train and test
X_train = X[:int(len(X)*0.8)]
y_train = y[:int(len(y)*0.8)]
X_test = X[int(len(X)*0.8):]
y_test = y[int(len(y)*0.8):]
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# train model until convergence
history = critic.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), verbose=0, batch_size=64, callbacks=[early_stopping, model_checkpoint], shuffle=True)
critic.load_weights('best_model_weights.h5')
# print val accuracy and loss
print(f'Val accuracy: {history.history["val_accuracy"][-1]}')
print(f'Val loss: {history.history["val_loss"][-1]}')
print(f'Mean of y_test: {np.mean(y_test)}')

In [ ]:
# plot accuracy 
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# plot loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### critic in real-time game

In [ ]:
player1 = MCTSPlayer(maxIters=100, maxTime=1000, argmax=True)
player2 = MCTSPlayer(maxIters=100, maxTime=1000, argmax=True)

game = HexGame(None, None, boardSize, plot=False)
turn = 0
while not game.isTerminal():
    print(critic.predict(game.getNNState(), verbose=0)[0][0])
    if turn % 2 == 0:
        player1.playAction(game)
    else:
        player2.playAction(game)
    turn += 1
print("winner is: ", game.getResult())

### test how good on last move

In [ ]:
player1 = MCTSPlayer(maxIters=100, maxTime=1000, argmax=True)
player2 = MCTSPlayer(maxIters=100, maxTime=1000, argmax=True)

correct = 0
for _ in range(20):
    game = HexGame(None, None, boardSize, plot=False)
    turn = 0
    while not game.isTerminal():
        lastState = game.getNNState()
        if turn % 2 == 0:
            player1.playAction(game)
        else:
            player2.playAction(game)
        turn += 1
    pred = critic.predict(lastState, verbose=0)[0][0]
    print(f'Prediction: {pred}, Actual: {game.getResult()}')
    if pred > 0.5 and game.getResult() == 1:
        correct += 1
    elif pred < 0.5 and game.getResult() == -1:
        correct += 1

print(f'Accuracy: {correct/20}')

### Sequential games comparison

In [ ]:
rounds = 4
# model = createModel(size=boardSize)
model = loadModel(f'model.{boardSize}')
neuralNetPlayer = NeuralNetPlayer(model=model, argmax=True)
randomPlayer = RandomPlayer()
tournament = Tournament(HexGame, neuralNetPlayer, randomPlayer, boardSize=boardSize, plot=True)
tournament.run(rounds)
wins, losses, draws = tournament.getResults()
print(f"Neuralnet Player: {wins} wins, {losses} losses, {draws} draws")

replay = nnMctsPlayer.mcts.replayBuffer
# TODO: flip both axis and double the replay buffer
print(f'Length of replay buffer: {len(replay)}')

In [ ]:
replay

In [ ]:
# train model on replay buffer
X = np.array([x[0] for x in replay]).reshape(len(replay), boardSize*boardSize)
y = np.array([x[1] for x in replay]).reshape(len(replay), boardSize*boardSize)
model.fit(X, y, epochs=10, verbose=1)

In [ ]:
# create new game
game = HexGame(None, None, size=3)
board = game.getNNState()
# prediction = model.predict(board)

# plot distribution of actions predictions of empty board
# plt.scatter(range(len(prediction[0])), prediction[0])

# plot distribution actions of empty board with mcts
mc = Mcts(maxIters=5000, maxTime=15)
mc.search(game)
dist = mc.replayBuffer
plt.scatter(range(len(dist[0][-1])), dist[0][-1])
plt.show()

In [ ]:
# TODO: check who wins by counting 1's and -1's in last layer of y

In [ ]:
s = 5
dataName = f'replayBuffer{s}.pickle'
with open(dataName, 'rb') as f:
    replay = pickle.load(f)

X = np.array([x[0] for x in replay]).reshape(len(replay),-1)
y = np.array([x[1] for x in replay]).reshape(len(replay),-1)

In [ ]:
model = createModel(size=s)

In [ ]:
games = [game.getNNState()[0] for _ in range(10)]

In [ ]:
games

In [ ]:
games = np.array(games)

In [ ]:
games[0]

In [ ]:
X[:10]

In [ ]:
y[:10]

### Load replaybuffer

In [ ]:
dataName = f'replayBuffer{boardSize}.pickle'
with open(dataName, 'rb') as f:
    replay = pickle.load(f)

### Create training data and train

In [ ]:
X = np.array([x[0] for x in replay]).reshape(len(replay), boardSize*boardSize)
y = np.array([x[1] for x in replay]).reshape(len(replay), boardSize*boardSize)
print(X.shape)
print(y.shape)

numModels = 5
models = []

for i in range(numModels):
    newModel = tf.keras.models.clone_model(model)
    newModel.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.01))
    newModel.fit(X, y, epochs=20, verbose=0)
    models.append(newModel)

### Test vs random

In [ ]:
nn_better = 0
total_wins = []
for i in range(numModels):
    tournament = Tournament(HexGame, NeuralNetPlayer(model=models[i]), RandomPlayer(), boardSize=boardSize)
    tournament.run(11)
    wins, losses, draws = tournament.getResults()
    total_wins.append(wins)
    if wins > losses:
        nn_better += 1
    print(f"Model {i} vs random: {wins} wins, {losses} losses, {draws} draws")

    tournament = Tournament(HexGame, NeuralNetPlayer(model=models[i]),  MCTSPlayer(maxIters=50, maxTime=20), boardSize=boardSize)
    tournament.run(11)
    wins, losses, draws = tournament.getResults()
    total_wins.append(wins)
    if wins > losses:
        nn_better += 1
    print(f"Model {i} vs mcts: {wins} wins, {losses} losses, {draws} draws")

print(f"NN MCTS Player: {nn_better} models better than random player")

In [ ]:
plt.hist(total_wins)
print(f'winrate: {int(100*sum(total_wins)/len(total_wins)/11)}%')

### Visualize a model

In [ ]:
model = models[0]
# use %magic to make plots pop up in a separate window
tournament = Tournament(HexGame, NeuralNetPlayer(model=model), RandomPlayer(), boardSize=boardSize, plot=True)
tournament.run(1)

### Save model

In [ ]:
# model.save('model.h5')